# Business Problem

One of the most important problems in e-commerce is the correct calculation of the points given to the products after the sale.
The solution to this problem means more customer satisfaction for the e-commerce site, product prominence for sellers and a smooth shopping experience for buyers. Another problem is the correct ranking of the reviews given to the products. Since the prominence of misleading reviews will directly affect the sales of the product, it will cause both financial loss and customer loss. In solving these 2 basic problems, e-commerce sites and sellers will increase their sales while customers will complete their purchasing journey smoothly.

# Data Story

- Amazon ürün verilerini içeren bu veri seti ürün kategorileri ile çeşitli metadataları içermektedir.
- Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

**Variables
- **reviewerID**: User ID
- **asin:** Product ID
- **reviewerName:** Username
- **helpful:** Useful evaluation rating
- **reviewText:** Review
- **overall:** Product rating
- **summary:** Evaluation summary
- **unixReviewTime:** Review time
- **reviewTime:** Review time Raw
- **day_diff:** Number of days since evaluation
- **helpful_yes:** Number of times the evaluation was found helpful
- **total_vote:** Number of votes for the evaluation

In [1]:
# import Required Libraries

import numpy as np
import pandas as pd
import math
import scipy.stats as st
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Adjusting Row Column Settings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [3]:
# Loading the Data Set

df = pd.read_csv("/kaggle/input/amazon-review/amazon_review.csv")

In [4]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(4915, 12)
##################### Types #####################
reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object
##################### Head #####################
       reviewerID        asin  reviewerName helpful                                         reviewText  overall                                 summary  unixReviewTime  reviewTime  day_diff  helpful_yes  total_vote
0  A3SBTW3WS4IQSN  B007WTAJTO           NaN  [0, 0]                                         No issues.  4.00000                              Four Stars      1406073600  2014-07-23       138            0           0
1  A18K1ODH1I2MVB  B007WTAJTO          0mie  [0, 0]  Purchased this for my device, i

In [5]:
df['overall'].value_counts()

5.00000    3922
4.00000     527
1.00000     244
3.00000     142
2.00000      80
Name: overall, dtype: int64

In [6]:
df['overall'].mean()

4.587589013224822

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


# Calculating the Weighted Grade Point Average by Date.

In [8]:
df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean()

4.6957928802588995

In [9]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].quantile(0.50)), "overall"].mean()

4.636140637775961

In [10]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean()

4.571661237785016

In [11]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean()

4.4462540716612375

# Determination of time based average weights

In [12]:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100

In [13]:
time_based_weighted_average(df, w1=28, w2=26, w3=24, w4=22)

4.595593165128118

# Creating the helpful_no Variable

In [14]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [15]:
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]

In [16]:
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13


# Calculating score_pos_neg_diff, score_average_rating and wilson_lower_bound Scores and Adding to Data Set

In [17]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
     Calculate Wilson Lower Bound Score

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is considered as the WLB score.
    - The score is used for product ranking.
    - Note
    If the scores are between 1-5, 1-3 can be marked as negative and 4-5 as positive and can be made Bernoulli compatible.
    This brings some problems with it. For this reason, a Bayesian average rating is required.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score floa

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [18]:
def score_up_down_diff(up, down):
    return up - down


In [19]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [20]:
def score_average_rating(pos, neg):
    if pos + neg == 0:
        return 0
    return pos / (pos + neg)

# score_pos_neg_diff

In [21]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)


In [22]:
df.sort_values("score_pos_neg_diff", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37
315,"Amazon Customer ""johncrea""",5.00000,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28


# score_average_rating

In [23]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)


In [24]:
df.sort_values("score_average_rating", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000
435,Anthony L cate,5.00000,Love the extra storage,1,0,1,2012-07-24,1,1.00000
2901,luis,5.00000,Awesome and fast card :),1,0,1,2013-05-13,1,1.00000
2204,"jbwam ""jbwam""",2.00000,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.00000
2206,JCBiker,5.00000,Great card,1,0,1,2013-10-31,1,1.00000
3408,"Neng Vang ""Neng2012""",5.00000,working no problem,1,0,1,2013-07-25,1,1.00000


# wilson_lower_bound

In [25]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)


In [26]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52,0.88235,0.78465
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.80576,0.73214
3807,R. Heisler,3.00000,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.88000,0.70044
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37,0.78462,0.67033
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75229,0.66359
